In [11]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [7]:
all_errors = pd.read_csv('sentence_label_pred.tsv', sep='\t')

In [8]:
all_errors

,Sentence,Labels,Predictions
0,I’m really sorry about your situation :( Altho...,sadness,"love,sadness"
1,It's wonderful because it's awful. At not with.,admiration,NaN
2,"Kings fan here, good luck to you guys! Will be...",excitement,optimism
3,"I didn't know that, thank you for teaching me ...",gratitude,gratitude
4,They got bored from haunting earth for thousan...,neutral,neutral
...,...,...,...
5422,Thanks. I was diagnosed with BP 1 after the ho...,gratitude,gratitude
5423,Well that makes sense.,approval,neutral
5424,Daddy issues [NAME],neutral,neutral
5425,So glad I discovered that subreddit a couple m...,admiration,joy


In [5]:
with open('sentence_label_pred.tsv', 'r') as f:
    lines = [line for line in f.read().split('\n') if line][1:]

In [36]:
from collections import defaultdict
emo2badpred = defaultdict(list)

In [37]:
for line in tqdm(lines):
    sent, trueLabel, predLabel = line.split('\t')
    if trueLabel:
        all_pred_labels = predLabel.split(',')
        all_true_labels = trueLabel.split(',')
        for tl in all_true_labels:
            if tl not in all_pred_labels:
                note = "true: %s\npred: %s\n"%(tl, ', '.join(all_pred_labels))
                emo2badpred[tl].append(note + sent + '\n')

100%|██████████| 5427/5427 [00:00<00:00, 948278.95it/s]


In [38]:
len(emo2badpred)

28

In [54]:
empty_count = 0
for sent in emo2badpred['embarrassment']:
    if 'pred: \n' in sent:
        empty_count += 1

In [39]:
print('\n'.join(emo2badpred['embarrassment']))

true: embarrassment
pred: neutral
I was teased for being a virgin when I was a 6th grader- in 2005

true: embarrassment
pred: disgust
*10 minutes of awkward roleplay*

true: embarrassment
pred: 
209 comments and we don’t get to read any? All that good pasta advice going to waste. Shame.

true: embarrassment
pred: surprise
I am embarrassed to admit that I was shocked to find out that his books all had the same exact weight and that you knew what that weight was...

true: embarrassment
pred: disgust
And yet it looks like one thing you can't is read. Shame shame.

true: embarrassment
pred: 
Try nonchalantly handing them your card as if they had dropped it. I think its normal to be shy. *handing on exit, otherwise it could get awkward

true: embarrassment
pred: 
Yeah, I always thought [NAME] would be an improved [NAME]. It's a damn shame we haven't thrown to him more often.

true: embarrassment
pred: amusement, sadness
I felt bad for laughing

true: embarrassment
pred: curiosity, neutral
C

In [55]:
empty_count / len(emo2badpred['embarrassment'])

0.43243243243243246

In [34]:
len(emo2badpred['embarrassment'])

37

### Get label to training data

In [42]:
with open('../goemotions/data/train.tsv', 'r') as f:
    all_train_data = [line.split('\t') for line in f.read().split('\n')[1:] if line]

In [46]:
id2emo = {}
with open('../goemotions/data/emotions.txt','r') as f:
    emos = f.read().split('\n')
    for i, emo in enumerate(emos):
        id2emo[i] = emo

In [49]:
emo2trainData = defaultdict(list)
for line in all_train_data:
    emos = map(int, line[1].split(","))
    for emo in emos:
        emo2trainData[id2emo[emo]].append(line[0])

In [51]:
emo2trainData['curiosity']

['Ok, then what the actual fuck is your plan?',
 "It's crazy how far Photoshop has come. Underwater bridges?!! NEVER!!!",
 'I think the 90 day rule applies to increases over 5%?',
 'So this means the people who have debt can see those that don’t. Am I sensing an easier target for muggings and such?',
 'Who is this Wild team? Where have they been?',
 "Wow, that landlady sure is a piece of shit. I'd be interested in knowing how many people she has discriminated against or harassed in the past.",
 "Kinda. And thank you. I'm curious about those files and film and fotos you mentioned.",
 'Is this a [NAME] reference. I feel like it is.',
 'Hmmm, can I see that statistic claiming Rowan engineering is that high?',
 'Who do you think can lead the Packers to where they need to be?',
 'You live in Appalachia, don’t you?',
 'Also curious. But as someone who lives in northern Ontario, this is pretty accurate',
 'Like the handout this guy got to build the stadium?',
 'I’m assuming you’re referring t

In [52]:
shortcur_count = 0
for cur_sent in emo2trainData['curiosity']:
    if 'can' in cur_sent or '?' in cur_sent:
        shortcur_count += 1
print(shortcur_count / len(emo2trainData['curiosity']))

0.8201734367868553


### Analysis for `caring`

In [56]:
bad_caring = emo2badpred['caring']

In [59]:
print('\n'.join(bad_caring))

true: caring
pred: neutral
please dont bring a woman in as a sex toy for you both. hire a sex worker instead

true: caring
pred: admiration, love
I’m so proud of you for taking care of you and your babies, mama! We love you and stay strong.

true: caring
pred: neutral
All the ladies can stay at my place.

true: caring
pred: 
Hi, [NAME]! I thought I would stop by and wish you a wonderful and prosperous year! Have a good one! -HappyFriendlyBot

true: caring
pred: remorse
Just be honest with this person. Sorry I didn’t read the whole thing I don’t have time rn :(

true: caring
pred: neutral
Praying for me family.

true: caring
pred: neutral
Fascism doesn't even exist in America. [NAME], these kids are so sheltered. 

true: caring
pred: love, neutral
If [NAME] is being considered for all defense, [NAME] needs more love too

true: caring
pred: neutral
Let it go, look to the future, do not live in the past

true: caring
pred: 
Our father will protect us <3

true: caring
pred: 
If this thread

In [60]:
neutral_cross_count = 0
for sent in bad_caring:
    if 'neutral' in sent:
        neutral_cross_count += 1

In [61]:
neutral_cross_count / len(bad_caring)

0.45132743362831856

In [62]:
len(bad_caring)

113

### Analysis for `gratitude`

In [63]:
emo2trainData['gratitude']

['Yes I heard abt the f bombs! That has to be why. Thanks for your reply:) until then hubby and I will anxiously wait 😝',
 'Thank you friend',
 'Super, thanks',
 'aw, thanks! I appreciate that! ',
 'Thanks! I love watching him every week',
 'Very interesting. Thx',
 "Thanks DB, I'll see if I can find the book",
 'Awesome. Thanks!',
 'Awesome! I’m a cradle [RELIGION], so really interesting to hear your experience. Thanks for sharing.',
 'Thank you for at least doing something good sir.',
 "Thanks I'll give it a go. Nothing used for randoms at all? Would it be better for me to find a group to play with?",
 'Tournament was a blast met some really cool people loved it all. Thanks for holding it. Would love to do it again someday',
 "Yeah, I am! I'll private message you a link to my profile. Thank you.",
 "Kinda. And thank you. I'm curious about those files and film and fotos you mentioned.",
 'Glad to see Hearts fans giving [NAME] such a warm welcome back..',
 'Welcome to the party pal!',


In [67]:
thank_you_count = 0
for grat_sent in emo2trainData['gratitude']:
    if 'thank' in grat_sent.lower():
        thank_you_count += 1

In [68]:
thank_you_count / len(emo2trainData['gratitude'])

0.8471074380165289

In [70]:
print('\n'.join(emo2badpred['gratitude']))

true: gratitude
pred: excitement, joy
Happy cake day to him

true: gratitude
pred: optimism
I believe that you can search for providers here. Good luck and good on ya.

true: gratitude
pred: admiration
Holy smokes, good jobs!!!

true: gratitude
pred: admiration
Of course man, I really appreciate this. But it's okay, I can just message you through here. It's fine:)

true: gratitude
pred: admiration
Where can you watch community? I can’t find it anywhere and it looks great.

true: gratitude
pred: caring, optimism
Meditate. You may need to seek medical help. Good luck, friend.

true: gratitude
pred: joy
So he is half decent. Glad to see your finally making some sense.

true: gratitude
pred: amusement, joy
I'm glad you did because this was the comment where I stopped my fast scroll haha.

true: gratitude
pred: admiration, amusement
Lol that’s incredible! PLO is such an amazing game, so much more entertaining than NLH. Good job pal. 

true: gratitude
pred: admiration
oh yah, no worries. I a

In [71]:
bad_with_thank = 0
for bad_grat in emo2badpred['gratitude']:
    if 'thank' in bad_grat.lower():
        bad_with_thank += 1
bad_with_thank

0